In [ ]:
import torch
import os
from tools import train_net_shapenet

class args:
    config_file = "configs/shapenet/voxmesh_R50.yaml"
    opts = ""
    num_gpus = 0
    data_dir = 'ShapeNetV1processed'
    copy_data = False
    no_color = True
    
cfg = train_net_shapenet.setup(args())
loaders = train_net_shapenet.setup_loaders(cfg)    
    

In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
#keras.models
model = models.Sequential()
model.add(layers.Input(shape=(24, 48, 48, 48, 1)))
model.add(layers.ConvLSTM3D(
    filters=1,                           # number kernel
    kernel_size=(9, 9, 9),                # shape kernel
    padding="same",                      # "valid" or "same"
    strides=(1, 1, 1),                    # skip kernel 
    dilation_rate=(1, 1, 1),              # skip noron 
    return_sequences=True,               # output all noron if True
    return_state=False,                   # True --> output + last state
    go_backwards=False,                   # if go_backwards: --> inputs = reverse(inputs, 0)
    stateful=False,                       # save state inloop --> but bachsizi input layer
    activation="tanh",                    # relu better 
    recurrent_activation="hard_sigmoid",  # dont touch :D 
    # Layer weight customize -->
    use_bias=False,
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    # dropout --> not good
    dropout=0.0,
    recurrent_dropout=0.0,
    data_format='channels_last',
))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
model.summary()

In [ ]:
log_dir = "/home/alan/Desktop/meshrcnn/log/"

In [ ]:
def rotateback(voxels, P):
    voxel_coords = []
    for i, cur_voxels in enumerate(voxels):              
        cur_voxel_coords = loaders["train"].dataset._coordinates(cur_voxels, P[i].inverse())
        voxel_coords.append(cur_voxel_coords) 
    voxel_coords = tuple(voxel_coords)
    
    voxels = []
    for i, cur_voxel_coords in enumerate(voxel_coords):
        cur_voxels = loaders["train"].dataset._voxelize(cur_voxel_coords, P[0])
        voxels.append(cur_voxels)
    voxels = torch.stack(voxels, dim=0)
    
    return voxels

In [ ]:
from os.path import exists
x = torch.tensor(())
y = torch.tensor(())
device = torch.device("cpu")

checkpoint_path = "/home/alan/Desktop/lstm3dtest/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

missed = []

for i, batch in enumerate(loaders["train"]):

    _, _, _, _, _, Ps, id_strs = batch    
    batch = loaders["train"].postprocess(batch, device)
    imgs, meshes_gt, points_gt, normals_gt, voxels_gt = batch
    
    #voxels_gt = rotateback(voxels_gt.cpu(), Ps.cpu())     
    voxels_gt = voxels_gt.float()  

    words = id_strs[0].split('-')
    output_dir = "/home/alan/Desktop/meshrcnn_s/meshrcnn_s/datasets/shapenet/predictedVoxel"
    output_dir = os.path.join(output_dir, words[0], words[1], "predictedVoxels.pt")
    if not exists(output_dir):
        missed.append(output_dir)
        continue
    
    voxel_scores = (torch.load(output_dir).sigmoid()>0.7).float()
    assert(len(voxel_scores) == 24)
 
    voxel_scores = rotateback(voxel_scores.cpu(), Ps.cpu())   

    voxel_scores = voxel_scores[None, :, :, :, :, None]
    
    voxels_gt = voxels_gt[None, :, :, :, :, None]    

    y = torch.cat((y, voxels_gt.cpu()), 0)
    x = torch.cat((x, voxel_scores.cpu()), 0) 

    if len(x) == 8:
        history = model.fit(x.detach().numpy(), y.detach().numpy(), 
                            batch_size=10,epochs=10000, verbose=1, validation_split=0.2, 
                            callbacks=[cp_callback, tensorboard_callback])       
        x = torch.tensor(())
        y = torch.tensor(())

In [ ]:
#print(missed)

In [ ]:
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

checkpoint_path = "/home/alan/Desktop/lstm3dtest/cp.ckpt"
model.load_weights(checkpoint_path)
 
for i, batch in enumerate(loaders["train"]):
    _, _, _, _, _, Ps, id_strs = batch     
    batch = loaders["train"].postprocess(batch, 'cpu')
    imgs, meshes_gt, points_gt, normals_gt, voxels_gt = batch
    #voxels_gt_r = rotateback(voxels_gt.cpu(), Ps.cpu())
    voxels_gt = voxels_gt.float()  
    
    words = id_strs[0].split('-')
    mesh_rcnn_predicted_voxels_dir = "/home/alan/Desktop/meshrcnn_s/meshrcnn_s/datasets/shapenet/predictedVoxel"
    mesh_rcnn_predicted_voxels_dir = os.path.join(mesh_rcnn_predicted_voxels_dir, words[0], words[1], "predictedVoxels.pt")
    mesh_rcnn_predicted_voxels = (torch.load(mesh_rcnn_predicted_voxels_dir).sigmoid()>0.7).float()
    mesh_rcnn_predicted_voxels = rotateback(mesh_rcnn_predicted_voxels.cpu(), Ps.cpu()).float()
    mesh_rcnn_predicted_voxels_input = mesh_rcnn_predicted_voxels[None, :, :, :, :, None]
    print("voxels_gt.shape")
    print(voxels_gt.shape)
    print("mesh_rcnn_predicted_voxels_input.shape")
    print(mesh_rcnn_predicted_voxels_input.shape)
    result = model.predict(mesh_rcnn_predicted_voxels_input.cpu().detach().numpy())
    #result = (torch.from_numpy(np.squeeze(result))>0.7).float()
    result = (torch.from_numpy(np.squeeze(result))).float()
    print("result.shape")
    print(result.shape)
    
    fig = make_subplots(
    rows=6, cols=3,
    specs=[[{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}],           
           [{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}]])    

    # y_true = [[0, 1], [0, 0]]
    # y_pred = [[-18.6, 0.51], [2.94, -12.8]]
    # Using default 'auto'/'sum_over_batch_size' reduction type.
    bce = tf.keras.losses.BinaryCrossentropy()
    # print(bce(y_true, y_pred).numpy())    
    
    for index in range(0, 6):
      
        voxels_gt_np = voxels_gt[index].cpu().numpy()
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxels_gt_np.flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
                ),
            row=index + 1,
            col=1,
        )   

        print(np.histogram(voxels_gt_np.flatten()))
 
        voxel_probs_np = mesh_rcnn_predicted_voxels[index].cpu().numpy()
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(           
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxel_probs_np.flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
            ),
            row=index + 1,
            col=2,        
        )
        
        print(np.histogram(voxel_probs_np.flatten()))

        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]        
        fig.add_trace( 
            go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=result[index].numpy().flatten(),
            isomin=0,
            isomax=1,
            opacity=0.1, # needs to be small to see through all surfaces
            surface_count=20, # needs to be a large number for good volume rendering
            ),
            row=index + 1,
            col=3,        
        )
        
        print(np.histogram(result[index].flatten()))
    
    #fig.show()
    
    #mesh_rcnn_predicted_voxels = rotateback(mesh_rcnn_predicted_voxels.cpu(), Ps.cpu()) 
    bce = tf.keras.losses.BinaryCrossentropy()    
    print("index")
    print(index)
    print(bce(voxels_gt[index].cpu().detach().numpy(), mesh_rcnn_predicted_voxels[index].cpu().detach().numpy()).numpy()) 
    print(bce(voxels_gt[index].cpu().detach().numpy(), result[index]).numpy())

    
    break

In [ ]:
2c96b939342fcda4360c528c0fc0d777
2c981b96364b1baa21a66e8dfcce514a
2ca59da312d06381b927782fc69a1fbb

# European Conference on Computer Vision

In [ ]:

ConvLSTM3D
0.0037270044



